# Pre-class Reading: Day 19 (Nov 20, 2023)<br>Animations
Learning goals
1. Create animations using `matplotlib.animation.ArtistAnimation` and `matplotlib.animation.FuncAnimation`
1. Embed these animations into a notebook so they are viewable after the cell has been executed or save these animations to an HTML file


## *19.1 Overview of Animations in Jupyter*

It is often helpful to be able to view the time evolution of phenomena as an animation. Matplotlib has facilities for doing this. The page https://matplotlib.org/stable/api/animation_api.html has some examples.

We will look at two types of animations

1. Animating a list consisting of a bunch of images in sequence using `ArtistAnimation()`, and
1. Animating an object updated through a user function, using `FuncAnimation()`. 

Both are methods of the `matplotlib.animation package`, and are documented at the same link above. 

## *19.2 `matplotlib.animation.ArtistAnimation`*

The following script generates a sequence of 60 images of a sin wave progressing through one full cycle. It does this by saving a list of 60 images (plots in our example) and bundling them into an `ArtistAnimation` object. Within Jupyter, the easiest way to interact with this object is to convert it to a javascript html object using the `.to_jshtml()` method, which we can then embed directly in the notebook for viewing or save as an html file to watch outside of the notebook. Do not try to do more than one animation in any one notebook cell. Also note that embedding animations directly into the notebook will quickly make your notebooks have larger filesizes than you are used to.

The code below is heavily commented to help make sense of a lot of a lot of the coding choices that are particular to generating animations this way. 

Note that the cell below will take 5-10 seconds to run. The step of creating the animation object using `ArtistAnimation` is VERY slow and it is not the for loop making things slow.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Our main function for generating the animations
import matplotlib.animation as animation

# This will allow us to display our HTML video as output from a code cell
from IPython.display import HTML


### Our code for generating the list of plots to animate ###

# We will use a total of n_frames in our animation
n_frames = 60

# x-value from 0 to 2pi to plot one full wavelength of sin
x = np.linspace(0, 2 * np.pi, 100)

# Initialize the plotting canvas so that we can update our plot
# each frame instead of making an entirely new canvas each frame
fig, ax = plt.subplots()

# Initialize the "frames" list, where we will place each of our images/plots
frames = []

# Our main loop to create n_frames frames
for i_frame in range(n_frames):
    
    # Cycle the offset from 0 to 2pi during n_frames
    x_offset = i_frame/n_frames * 2*np.pi
    
    # Generate sin data using the offset
    y = np.sin(x + x_offset)
    
    # Plot the current data as "line"
    # - We discuss the "line," notation below
    line, = ax.plot(x, y, color='blue')
    
    # Add the current line plot to the frames list to animate later
    # - Note that ArtistAnimation expects a list of lists, which is why we use
    #   "frames.append([line])" and not "frames.append(line)"
    frames.append([line])  

# Create the animation using ArtistAnimation
# - blit = True forces the animation to only save information that changes from one
#   frame to the next instead of saving all of the static information (e.g., axes)
#   during each frame
ani = animation.ArtistAnimation(fig, frames, blit=True)

# Close the current plot to prevent an unwanted additional plot from being made when
# the cell has completed execution
# - This must come before "ani.to_jshtml" and "HTML(ani_html)" or else the animations
#   won't get displayed
plt.close() 

# Convert to our HTML javascript object
# - fps stands for frames per second and controls how fast the animation runs
# - default_mode can be set to once, loop or reflect
ani_html = ani.to_jshtml(fps=30, default_mode='loop') 

# Display the animation
HTML(ani_html)

### 19.2.1 Saving the animation to file

Our `ani_html` animation object is still stored in memory from the code above, so if we wanted to save it to file, we could easily do so. This will save a file to the same directory as this notebook that you should be able to open and view.

In [ ]:
with open("sin_animation.html", "w") as file:
    file.write(ani_html)

### 19.2.2 Your turn #1

There are a few parameters in the code above that are worth understanding a bit better. 

1. In the line `ani_html = ani.to_jshtml(fps=30, default_mode='loop')`, try changing the fps to different values. In the code above we created 60 frames and ran them at 30 frames per second. 
1. Try changing the `default_mode` to `once` and `reflect` and notice what happens when you run the animation
1. The object created by `ax.plot(x, y, color='blue')` is a nested object, potentially containing multiple elements. Thus, we use `line,` as a way to grab only the first element when we create each "frame".  Test what happens if we didn't use the comma, so  `line = ax.plot(x, y, color='blue')`. Alternatively, we could also access the first element using `line = ax.plot(x, y, color='blue')[0]`. Try getting it to work this way.
1. Test what happens if we don't specify a color `color='blue'` as an argument for our `ax.plot` function.

## *19.3 `matplotlib.animation.FuncAnimation`*

As an alternative to `ArtistAnimation`, we can use `FuncAnimation`, which takes over the responsibility of running the iteration. Here we need to create an initial plot similar to what we did before. 
```python
fig, ax = plt.subplots()
line, = ax.plot(x, np.sin(x))
```
But in the case of `FuncAnimation` we build a function (`update_line` in our example) to tell `FuncAnimation` how to update our plot. Similar to `solve_ivp` having its own internal iteration and calling our user-defined function each iteration, `FuncAnimation` will call our user-defined function a number of times specified by the user, which is `n_frames` in our example. 

We also introduce a slightly different way of updating only the elements of the plot which need to be updated. The example below uses `line.set_ydata(y)` to update the y-data in our line object each time `update_line` is called. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Function to be called by FuncAnimation to update the image
def update_line(i_frame):

    # Cycle the offset from 0 to 2pi during n_frames
    x_offset = i_frame/n_frames * 2*np.pi
    
    # Generate sin data using the offset
    y = np.sin(x + x_offset)    
    
    # Update the plot by updating only the ydata
    line.set_ydata(y)

# We will use a total of n_frames in our animation
n_frames = 60

# x-value from 0 to 2pi to plot one full wavelength of sin
x = np.linspace(0, 2 * np.pi, 100)

# Initialize the plotting canvas
fig, ax = plt.subplots()

# Create an initial line object to be updated by FuncAnimation
line, = ax.plot(x, np.sin(x))

# Create an FuncAnimation object ani
# - fig is the figure object being updated
# - update_line is the name of the function that updates fig
# - n_frames is an integer and equivalent to a for loop "for i_frame in range(n_frames)" 
#   where an updated value of i_frame is passed to update_line during each iteration
ani = animation.FuncAnimation(fig, update_line, n_frames)

plt.close() 

ani_html = ani.to_jshtml(fps=30) 
HTML(ani_html)

## *19.4 More examples*

Have a careful look at the examples below and see if you can figure out how they each work.

The following example shows how you can use `FuncAnimation` to generate animations which involve calculations in a separate function for each iteration. Similar to the line example above, we will use, and update, an instance of `scatter()`, i.e., a scatter plot. The initial plot is a dot located at `xpos`, `ypos`, and its position is updated using the method `set_offsets` of the scatter plot instance `im`. The function `newypos()` updates the position of the dot, shifting vertically by an amount `dypos`:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

n_frames = 50
xpos, ypos = 0.5, 0.
dypos = 1./50.

def update_point(num):
    global ypos
    ypos = ypos + dypos
    im.set_offsets((xpos, ypos))

fig, ax = plt.subplots()
plt.xlim(0, 1)
plt.ylim(0, 1)
im = ax.scatter(xpos, ypos)

ani = animation.FuncAnimation(fig, update_point, n_frames)

plt.close() 

ani_html = ani.to_jshtml(fps=30) 
HTML(ani_html)

The last example is similar, but now animates a scatter plot with many points. It uses some additional functions (`numpy.vstack`) and methods of a scatter plot instance (e.g. `set_sizes()` and `set_facecolor()`).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

npoint = 30
n_frames = 100
dstep = 400.

def update_point(i_frame):
    newx = x + np.random.randn(npoint) / dstep
    newy = y + np.random.randn(npoint) / dstep
    data = np.stack((newx, newy), axis=-1)
    im.set_offsets(data)

x = np.random.random(npoint)
y = np.random.random(npoint)
s = 500 * np.random.random(npoint)
c = np.random.random(npoint * 3).reshape(npoint, 3)

fig, ax = plt.subplots()
plt.ylim(-0.2, 1.2)
plt.xlim(-0.2, 1.2)
im = ax.scatter(x, y)
im.set_facecolor(c)
im.set_sizes(s)

ani = animation.FuncAnimation(fig, update_point, n_frames)

plt.close() 

ani_html = ani.to_jshtml(fps=30) 
HTML(ani_html)

## *Submitting this reading assignment*
Before submitting your work, restart + rerun your notebook to make sure that your self-assessmet questions run correctly and without error. After you've executed and checked your notebook, choose: File >> Save_and_Export_Notebook_As >> HTML. This will download an HTML version of your notebook to your computer which you can upload to Canvas